In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ariyoomotade/netflix-data-cleaning-analysis-and-visualization")

print("Path to dataset files:", path)
# The path variable already points to the directory where netflix1.csv should be located.
# So, just use the filename directly.
data=pd.read_csv(path+"/netflix1.csv")

100%|██████████| 270k/270k [00:00<00:00, 70.1MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/ariyoomotade/netflix-data-cleaning-analysis-and-visualization/versions/1


In [6]:
data.head()

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,Documentaries
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries"
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,"Children & Family Movies, Comedies"
4,s8,Movie,Sankofa,Haile Gerima,United States,9/24/2021,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies"


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8790 non-null   object
 1   type          8790 non-null   object
 2   title         8790 non-null   object
 3   director      8790 non-null   object
 4   country       8790 non-null   object
 5   date_added    8790 non-null   object
 6   release_year  8790 non-null   int64 
 7   rating        8790 non-null   object
 8   duration      8790 non-null   object
 9   listed_in     8790 non-null   object
dtypes: int64(1), object(9)
memory usage: 686.8+ KB


Converting date_added to datetime

In [8]:
data['date_added']=pd.to_datetime(data['date_added'])

In [9]:
data.nunique()

,0
show_id,8790
type,2
title,8787
director,4528
country,86
date_added,1713
release_year,74
rating,14
duration,220
listed_in,513


Checking For Missing and duplicate values

In [10]:
# get the number of missing data points per column
missing_values=data.isnull().sum()
missing_values

,0
show_id,0
type,0
title,0
director,0
country,0
date_added,0
release_year,0
rating,0
duration,0
listed_in,0


In [11]:
data.duplicated().sum()

np.int64(0)

No Duplicates and null values so data is clean

Checking for Inconsistences in duration column:

In [12]:
data[['type','duration']].value_counts()

type     duration  
TV Show  1 Season      1791
         2 Seasons      421
         3 Seasons      198
Movie    90 min         152
         93 min         146
                       ... 
         228 min          1
         8 min            1
         9 min            1
TV Show  17 Seasons       1
         11 Seasons       1
Name: count, Length: 220, dtype: int64

In [13]:
data[(data["type"] == "TV Show") & (data["duration"].str.contains("min"))]

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in


In [14]:
data[(data["type"] == "Movie") & (data["duration"].str.contains("Seasons"))]

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in


Splitting duration into Movie_duration and Number_of_seasons

In [15]:
data['movie_duration'] = data['duration'].str.extract(r'(\d+) min').fillna(0).astype(int)
data['num_seasons'] = data['duration'].str.extract(r'(\d+) Season').fillna(0).astype(int)
data.drop(columns=['duration'], inplace=True)

Checking for Duplicate Movie Titles

In [16]:
data['title'].value_counts()

,count
title,
9-Feb,2
15-Aug,2
22-Jul,2
"Septiembre, un llanto en silencio",1
Set Up,1
...,...
Sei,1
Superlopez,1
Oversize Cops,1


Convert categorical columns to category type:

In [17]:
categorical_columns=['type','country','rating']
data[categorical_columns]=data[categorical_columns].astype('category')

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   show_id         8790 non-null   object        
 1   type            8790 non-null   category      
 2   title           8790 non-null   object        
 3   director        8790 non-null   object        
 4   country         8790 non-null   category      
 5   date_added      8790 non-null   datetime64[ns]
 6   release_year    8790 non-null   int64         
 7   rating          8790 non-null   category      
 8   listed_in       8790 non-null   object        
 9   movie_duration  8790 non-null   int64         
 10  num_seasons     8790 non-null   int64         
dtypes: category(3), datetime64[ns](1), int64(3), object(4)
memory usage: 578.8+ KB


Checking Duplicate titiles:

In [19]:
data['title'].value_counts()

,count
title,
9-Feb,2
15-Aug,2
22-Jul,2
"Septiembre, un llanto en silencio",1
Set Up,1
...,...
Sei,1
Superlopez,1
Oversize Cops,1


Remove Suspicious titles:

In [20]:
suspicious_titles=data[data['title'].str.match(r"\d{1,2}-[A-Za-z]{3}")]
data.drop(suspicious_titles.index, inplace=True)

Handling Inconsistences in rating column:

In [21]:
data['rating'].value_counts()

,count
rating,
TV-MA,3205
TV-14,2152
TV-PG,861
R,797
PG-13,490
TV-Y7,333
TV-Y,306
PG,287
TV-G,220


In [22]:
data['rating']=data['rating'].cat.add_categories(['Not_rated'])
data.loc[data['rating'].isin(['UR','NR']), 'rating']='Not_rated'

In [23]:
#remove unused categories (UR, NR)
data['rating']=data['rating'].cat.remove_unused_categories()

Replacing director names with unknown:

In [24]:
data['director']=data['director'].replace('Not Given', 'Unknown')

Dataset Statistics:

In [25]:
data.describe(include='all')

,show_id,type,title,director,country,date_added,release_year,rating,listed_in,movie_duration,num_seasons
count,8783,8783,8783,8783,8783,8783,8783.000000,8783,8783,8783.000000,8783.000000
unique,8783,2,8783,4527,86,NaN,NaN,13,513,NaN,NaN
top,s8786,Movie,YOM,Unknown,United States,NaN,NaN,TV-MA,"Dramas, International Movies",NaN,NaN
freq,1,6121,1,2586,3240,NaN,NaN,3205,362,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,2019-05-17 23:14:25.262438656,2014.180348,NaN,NaN,69.380849,0.531140
min,NaN,NaN,NaN,NaN,NaN,2008-01-01 00:00:00,1925.000000,NaN,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,2018-04-06 00:00:00,2013.000000,NaN,NaN,0.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,2019-07-04 00:00:00,2017.000000,NaN,NaN,88.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,2020-08-20 00:00:00,2019.000000,NaN,NaN,106.000000,1.000000
max,NaN,NaN,NaN,NaN,NaN,2021-09-25 00:00:00,2021.000000,NaN,NaN,312.000000,17.000000


Splitting Listed_in column into multiple categories:

In [26]:
data["listed_in"]=data["listed_in"].apply(lambda x: re.split(r", | & ", x))

In [27]:
data["listed_in"]

,listed_in
0,[Documentaries]
1,"[Crime TV Shows, International TV Shows, TV Ac..."
2,"[TV Dramas, TV Horror, TV Mysteries]"
3,"[Children, Family Movies, Comedies]"
4,"[Dramas, Independent Movies, International Mov..."
...,...
8785,"[International TV Shows, TV Dramas]"
8786,[Kids' TV]
8787,"[International TV Shows, Romantic TV Shows, TV..."
8788,[Kids' TV]


Renaming Columns:

In [30]:
data.rename(columns={"date_added":"date", "listed_in":"category"}, inplace=True)

Saving the cleaned dataset:

In [34]:
data.to_csv('Netflix_cleaned.csv', index=False)